In [7]:
import copy

class GameState:
    def __init__(self, board, current_player, en_passant_target):
        self.board = copy.deepcopy(board)
        self.current_player = current_player
        self.en_passant_target = en_passant_target

class Piece:
    def __init__(self, colour):
        self.colour = colour

    def valid_moves(self, start, board):
        return []

    def __str__(self):
        return 'P'


class Pawn(Piece):
    def __str__(self):
        return '♙' if self.colour == 'white' else '♟'

    def valid_moves(self, start, board, en_passant_target=None):
        moves = []
        direction = -1 if self.colour == 'white' else 1
        row, col = start

        if board.is_empty((row + direction, col)):
            moves.append((row + direction, col))

            if (self.colour == 'white' and row == 6) or (self.colour == 'black' and row == 1):
                if board.is_empty((row + 2 * direction, col)):
                    moves.append((row + 2 * direction, col))

        for dc in [-1, 1]:
            new_row, new_col = row + direction, col + dc
            if board.in_bounds((new_row, new_col)):
                target = board.get_piece((new_row, new_col))
                if target and target.colour != self.colour:
                    moves.append((new_row, new_col))

        if en_passant_target:
            ep_row, ep_col = en_passant_target
            if row + direction == ep_row and abs(col - ep_col) == 1:
                moves.append((ep_row, ep_col))
        return moves


class Rook(Piece):
    def __str__(self):
        return '♖' if self.colour == 'white' else '♜'

    def valid_moves(self, start, board, en_passant_target=None):
        moves = []
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        row, col = start
        for dr, dc in directions:
            r, c = row, col
            while True:
                r += dr
                c += dc
                if not board.in_bounds((r, c)):
                    break
                if board.is_empty((r, c)):
                    moves.append((r, c))
                else:
                    if board.get_piece((r, c)).colour != self.colour:
                        moves.append((r, c))
                    break
        return moves


class Knight(Piece):
    def __str__(self):
        return '♘' if self.colour == 'white' else '♞'

    def valid_moves(self, start, board, en_passant_target=None):
        moves = []
        row, col = start
        deltas = [(2,1), (2,-1), (-2,1), (-2,-1), (1,2), (1,-2), (-1,2), (-1,-2)]
        for dr, dc in deltas:
            r, c = row + dr, col + dc
            if board.in_bounds((r, c)):
                if board.is_empty((r, c)) or board.get_piece((r, c)).colour != self.colour:
                    moves.append((r, c))
        return moves


class Bishop(Piece):
    def __str__(self):
        return '♗' if self.colour == 'white' else '♝'

    def valid_moves(self, start, board, en_passant_target=None):
        moves = []
        directions = [(1,1), (1,-1), (-1,1), (-1,-1)]
        row, col = start
        for dr, dc in directions:
            r, c = row, col
            while True:
                r += dr
                c += dc
                if not board.in_bounds((r, c)):
                    break
                if board.is_empty((r, c)):
                    moves.append((r, c))
                else:
                    if board.get_piece((r, c)).colour != self.colour:
                        moves.append((r, c))
                    break
        return moves


class Queen(Piece):
    def __str__(self):
        return '♕' if self.colour == 'white' else '♛'

    def valid_moves(self, start, board, en_passant_target=None):
        return Rook(self.colour).valid_moves(start, board) + Bishop(self.colour).valid_moves(start, board)


class King(Piece):
    def __str__(self):
        return '♔' if self.colour == 'white' else '♚'

    def valid_moves(self, start, board, en_passant_target=None):
        moves = []
        row, col = start
        for dr in [-1,0,1]:
            for dc in [-1,0,1]:
                if dr == 0 and dc == 0:
                    continue
                r, c = row + dr, col + dc
                if board.in_bounds((r, c)):
                    if board.is_empty((r, c)) or board.get_piece((r, c)).colour != self.colour:
                        moves.append((r, c))
        return moves


class Board:
    def __init__(self):
        self.grid = [[None for _ in range(8)] for _ in range(8)]
        self.setup_board()

    def setup_board(self):
        for colour, row_pawn, row_others in [('white', 6, 7), ('black', 1, 0)]:
            for i in range(8):
                self.grid[row_pawn][i] = Pawn(colour)
            self.grid[row_others][0] = Rook(colour)
            self.grid[row_others][7] = Rook(colour)
            self.grid[row_others][1] = Knight(colour)
            self.grid[row_others][6] = Knight(colour)
            self.grid[row_others][2] = Bishop(colour)
            self.grid[row_others][5] = Bishop(colour)
            self.grid[row_others][3] = Queen(colour)
            self.grid[row_others][4] = King(colour)

    def display(self):
        print("  a b c d e f g h")
        for i, row in enumerate(self.grid):
            print(8 - i, end=' ')
            for piece in row:
                print(str(piece) if piece else '.', end=' ')
            print(8 - i)
        print("  a b c d e f g h")

    def display_with_highlight(self, highlight_moves):
        print("   a  b  c  d  e  f  g  h")
        for i, row in enumerate(self.grid):
            print(8 - i, end=' ')
            for j, piece in enumerate(row):
                pos = (i, j)
                if pos in highlight_moves:
                    target = self.get_piece(pos)
                    if target:
                        # Вражеская фигура для взятия
                        print(f'({str(target)})', end='')
                    else:
                        # Свободное поле для хода
                        print(' * ', end='')
                else:
                    print(f' {str(piece) if piece else "."} ', end='')
            print(8 - i)
        print("   a  b  c  d  e  f  g  h")

    def in_bounds(self, pos):
        row, col = pos
        return 0 <= row < 8 and 0 <= col < 8

    def is_empty(self, pos):
        row, col = pos
        return self.grid[row][col] is None

    def get_piece(self, pos):
        row, col = pos
        return self.grid[row][col]

    def move_piece(self, start, end):
        sr, sc = start
        er, ec = end
        piece = self.grid[sr][sc]
        self.grid[er][ec] = piece
        self.grid[sr][sc] = None

    def find_king(self, colour):
        for row in range(8):
            for col in range(8):
                piece = self.grid[row][col]
                if isinstance(piece, King) and piece.colour == colour:
                    return (row, col)
        return None

    def is_under_attack(self, position, attacker_colour):
        for row in range(8):
            for col in range(8):
                piece = self.grid[row][col]
                if piece and piece.colour == attacker_colour:
                    moves = piece.valid_moves((row, col), self)
                    if position in moves:
                        return True
        return False


class Game:
    def __init__(self):
        self.board = Board()
        self.current_player = 'white'
        self.en_passant_target = None
        self.history = []
        self.save_state()

    def save_state(self):
        state = GameState(self.board, self.current_player, self.en_passant_target)
        self.history.append(state)

    def undo(self, steps=1):
        if len(self.history) <= steps:
            print("Cannot undo before game started.")
            return False
        for _ in range(steps):
            self.history.pop()
        last_state = self.history[-1]
        self.board = copy.deepcopy(last_state.board)
        self.current_player = last_state.current_player
        self.en_passant_target = last_state.en_passant_target
        print(f"Reverted {steps} move(s).")
        return True

    def parse_move(self, move):
        try:
            start_str, end_str = move.split()
            start = (8 - int(start_str[1]), ord(start_str[0]) - ord('a'))
            end = (8 - int(end_str[1]), ord(end_str[0]) - ord('a'))
            return start, end
        except:
            return None, None

    def check_status(self):
        king_pos = self.board.find_king(self.current_player)
        opponent = 'black' if self.current_player == 'white' else 'white'
        under_check = self.board.is_under_attack(king_pos, opponent)

        has_moves = False
        for row in range(8):
            for col in range(8):
                piece = self.board.get_piece((row, col))
                if piece and piece.colour == self.current_player:
                    valid = piece.valid_moves((row, col), self.board, self.en_passant_target)
                    for move in valid:
                        temp_board = self.copy_board()
                        temp_board.move_piece((row, col), move)
                        king_pos_temp = temp_board.find_king(self.current_player)
                        if not temp_board.is_under_attack(king_pos_temp, opponent):
                            has_moves = True
                            break
                    if has_moves:
                        break

        if under_check and not has_moves:
            print(f"Checkmate! {opponent.capitalize()} wins!")
            return True
        elif not under_check and not has_moves:
            print("Stalemate! It's a draw.")
            return True
        elif under_check:
            print(f"{self.current_player.capitalize()} is in check!")
            return False
        return False

    def copy_board(self):
        new_board = Board()
        for row in range(8):
            for col in range(8):
                piece = self.board.get_piece((row, col))
                if piece:
                    new_board.grid[row][col] = type(piece)(piece.colour)
                else:
                    new_board.grid[row][col] = None
        return new_board


    def play(self):
        while True:
            self.board.display()
            print(f"{self.current_player}'s move:")
            if self.check_status():
                break

            start_input = input("Select piece to move (e.g., e2), or 'undo', or 'undo N': ").strip()
            if start_input == "quit":
                break
            if start_input.startswith('undo'):
                parts = start_input.split()
                steps = 1
                if len(parts) == 2 and parts[1].isdigit():
                    steps = int(parts[1])
                self.undo(steps)
                continue

            if len(start_input) != 2 or start_input[0] not in 'abcdefgh' or start_input[1] not in '12345678':
                print("Invalid position.")
                continue


            start = (8 - int(start_input[1]), ord(start_input[0]) - ord('a'))
            piece = self.board.get_piece(start)
            if not piece or piece.colour != self.current_player:
                print("No valid piece at start position.")
                continue

            valid = piece.valid_moves(start, self.board, self.en_passant_target)
            legal_moves = []

            for move in valid:
                temp_board = self.copy_board()
                temp_board.move_piece(start, move)
                king_pos = temp_board.find_king(self.current_player)
                opponent = 'black' if self.current_player == 'white' else 'white'
                if not temp_board.is_under_attack(king_pos, opponent):
                    legal_moves.append(move)

            if not legal_moves:
                print("No valid moves for this piece.")
                continue

            print("Available moves highlighted (*), capturable opponent pieces shown in brackets:")
            self.board.display_with_highlight(legal_moves)

            end_input = input("Enter destination (e.g., e4), or 'cancel': ").strip()
            if end_input == 'cancel':
                continue
            if len(end_input) != 2 or end_input[0] not in 'abcdefgh' or end_input[1] not in '12345678':
                print("Invalid position.")
                continue

            end = (8 - int(end_input[1]), ord(end_input[0]) - ord('a'))
            if end not in legal_moves:
                print("Invalid move.")
                continue

            if isinstance(piece, Pawn) and end == self.en_passant_target:
                row, col = end
                capture_row = row + (1 if self.current_player == 'white' else -1)
                self.board.grid[capture_row][col] = None

            self.board.move_piece(start, end)

            if isinstance(piece, Pawn) and abs(start[0] - end[0]) == 2:
                intermediate_row = (start[0] + end[0]) // 2
                self.en_passant_target = (intermediate_row, start[1])
            else:
                self.en_passant_target = None

            self.save_state()

            self.current_player = 'black' if self.current_player == 'white' else 'white'


game = Game()
game.play()


  a b c d e f g h
8 ♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜ 8
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟ 7
6 . . . . . . . . 6
5 . . . . . . . . 5
4 . . . . . . . . 4
3 . . . . . . . . 3
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙ 2
1 ♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖ 1
  a b c d e f g h
white's move:
Select piece to move (e.g., e2), or 'undo', or 'undo N': e2
Available moves highlighted (*), capturable opponent pieces shown in brackets:
   a  b  c  d  e  f  g  h
8  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜ 8
7  ♟  ♟  ♟  ♟  ♟  ♟  ♟  ♟ 7
6  .  .  .  .  .  .  .  . 6
5  .  .  .  .  .  .  .  . 5
4  .  .  .  .  *  .  .  . 4
3  .  .  .  .  *  .  .  . 3
2  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙ 2
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖ 1
    a  b  c  d  e  f  g  h
Enter destination (e.g., e4), or 'cancel': quit
Invalid position.
  a b c d e f g h
8 ♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜ 8
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟ 7
6 . . . . . . . . 6
5 . . . . . . . . 5
4 . . . . . . . . 4
3 . . . . . . . . 3
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙ 2
1 ♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖ 1
  a b c d e f g h
white's move:
Select piece to move (e.g., e2), or 'undo', or 'undo N': quit
